In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bank-transaction-data/bank.xlsx


In [2]:
# pip install kagglehub[pandas-datasets]

import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = 'bank.xlsx'

df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "apoorvwatsky/bank-transaction-data",
    file_path,
)

print("First 5 records:")
display(df.head())


/tmp/ipykernel_17/448986686.py:8: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


First 5 records:


,Account No,DATE,TRANSACTION DETAILS,CHQ.NO.,VALUE DATE,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,.
0,409000611074',2017-06-29,TRF FROM Indiaforensic SERVICES,NaN,2017-06-29,NaN,1000000.0,1000000.0,.
1,409000611074',2017-07-05,TRF FROM Indiaforensic SERVICES,NaN,2017-07-05,NaN,1000000.0,2000000.0,.
2,409000611074',2017-07-18,FDRL/INTERNAL FUND TRANSFE,NaN,2017-07-18,NaN,500000.0,2500000.0,.
3,409000611074',2017-08-01,TRF FRM Indiaforensic SERVICES,NaN,2017-08-01,NaN,3000000.0,5500000.0,.
4,409000611074',2017-08-16,FDRL/INTERNAL FUND TRANSFE,NaN,2017-08-16,NaN,500000.0,6000000.0,.


In [3]:
def DataCleaning(df):

    # This is the copy of the data 
    processedData = df.copy()

    # Cleaning the data 
    print("Data Informations:\n",processedData.info())
    # Null values 
    print("\n\n\nDeposite null : ",processedData['DEPOSIT AMT'].isnull().sum()) 
    print("\nWithDrawal null :",processedData['WITHDRAWAL AMT'].isnull().sum())
    print("\nDATE null :",processedData['DATE'].isnull().sum())
    print("\nValue DATE null :",processedData['VALUE DATE'].isnull().sum())
    
    # Fill nulls (correct way)
    processedData['DEPOSIT AMT'] = processedData['DEPOSIT AMT'].fillna(0)
    processedData['WITHDRAWAL AMT'] = processedData['WITHDRAWAL AMT'].fillna(0)
    processedData['CHQ.NO.'] = processedData['CHQ.NO.'].fillna(0)
    
    # Feature Engineering
    processedData['ActualDay'] = processedData['VALUE DATE'].dt.day 
    processedData['ActualMonth'] = processedData['VALUE DATE'].dt.month
    processedData['ActualYear'] = processedData['VALUE DATE'].dt.year
    processedData = processedData.sort_values(by="VALUE DATE", ascending=False)

    # Droping unuseful data
    processedData = processedData.drop(columns=['Account No','.','VALUE DATE'])
    
    # Data Discribtions
    print(processedData.columns)
    print("\nDataset Size",processedData.shape)

    display(processedData.head())

    
    return processedData
processData = DataCleaning(df)
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116201 entries, 0 to 116200
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Account No           116201 non-null  object        
 1   DATE                 116201 non-null  datetime64[ns]
 2   TRANSACTION DETAILS  113702 non-null  object        
 3   CHQ.NO.              905 non-null     float64       
 4   VALUE DATE           116201 non-null  datetime64[ns]
 5   WITHDRAWAL AMT       53549 non-null   float64       
 6   DEPOSIT AMT          62652 non-null   float64       
 7   BALANCE AMT          116201 non-null  float64       
 8   .                    116201 non-null  object        
dtypes: datetime64[ns](2), float64(4), object(3)
memory usage: 8.0+ MB
Data Informations:
 None



Deposite null :  53549

WithDrawal null : 62652

DATE null : 0

Value DATE null : 0
Index(['DATE', 'TRANSACTION DETAILS', 'CHQ.NO.', 'WITHDRAWAL AMT',
  

,DATE,TRANSACTION DETAILS,CHQ.NO.,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,ActualDay,ActualMonth,ActualYear
116200,2019-03-05,Sweep Trf To: 40900036427,0.0,5000.0,0.0,-1.901417e+09,5,3,2019
27037,2019-03-05,MICRO ATM WDL FEE DT04031,0.0,0.0,195800.0,-5.393314e+08,5,3,2019
86355,2019-03-05,FDRL/INTERNAL FUND TRANSFE,0.0,0.0,300000.0,-1.686924e+09,5,3,2019
86356,2019-03-05,FDRL/INTERNAL FUND TRANSFE,0.0,300000.0,0.0,-1.687224e+09,5,3,2019
86357,2019-03-05,FDRL/INTERNAL FUND TRANSFE,0.0,300000.0,0.0,-1.687524e+09,5,3,2019


In [4]:
def DataDiscribtion(df):
    # Finding the unique data values
    total_day_counts = df['ActualDay'].value_counts()
    total_month_counts = df['ActualMonth'].value_counts()
    total_year_counts = df['ActualYear'].value_counts()

    # Over all discribtion
    display("Total list of days : ",total_day_counts.head())
    print("\n")
    display("Total list of months : ",total_month_counts.head())
    print("\n")
    display("Total list of year : ",total_year_counts.head())
    print("\n")
    # function wise distributions
    display(df.describe())

    # Less Then Zero balance
    count =0
    for value in df['BALANCE AMT']:
        if(value < 0):
            count = count +1;

    print("Minus Value in balance : ",count)
DataDiscribtion(processData)


'Total list of days : '

ActualDay
16    4806
27    4577
7     4517
9     4331
12    4279
Name: count, dtype: int64

'Total list of months : '

ActualMonth
7     11153
8     10832
9     10616
1     10510
12    10464
Name: count, dtype: int64

'Total list of year : '

ActualYear
2018    35534
2016    30373
2017    29112
2015    15658
2019     5524
Name: count, dtype: int64

,DATE,CHQ.NO.,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,ActualDay,ActualMonth,ActualYear
count,116201,116201.000000,1.162010e+05,1.162010e+05,1.162010e+05,116201.000000,116201.000000,116201.000000
mean,2017-05-20 00:08:40.477448448,6165.275049,2.068757e+06,2.052394e+06,-1.404852e+09,15.615907,6.680313,2016.869993
min,2015-01-01 00:00:00,0.000000,0.000000e+00,0.000000e+00,-2.045201e+09,1.000000,1.000000,2015.000000
25%,2016-05-30 00:00:00,0.000000,0.000000e+00,0.000000e+00,-1.690383e+09,8.000000,4.000000,2016.000000
50%,2017-06-05 00:00:00,0.000000,0.000000e+00,5.000000e+03,-1.661395e+09,15.000000,7.000000,2017.000000
75%,2018-05-26 00:00:00,0.000000,2.682500e+04,5.000000e+05,-1.236888e+09,23.000000,10.000000,2018.000000
max,2019-03-05 00:00:00,874525.000000,4.594475e+08,5.448000e+08,8.500000e+06,31.000000,12.000000,2019.000000
std,NaN,70854.929858,7.696881e+06,6.652164e+06,5.348202e+08,8.602524,3.491998,1.131124


Minus Value in balance :  113276


In [5]:
def DataPreProcess(df):
    processData = df.copy()
    processData['OverdraftAmount'] = (-df['BALANCE AMT']).clip(lower=0)
    display("Overdraft : ",processData['OverdraftAmount'])
    return processData;
processData = DataPreProcess(processData)    

'Overdraft : '

116200    1.901417e+09
27037     5.393314e+08
86355     1.686924e+09
86356     1.687224e+09
86357     1.687524e+09
              ...     
37591     1.588653e+09
37592     1.587953e+09
37582     1.585716e+09
37583     1.584916e+09
27046     5.399581e+08
Name: OverdraftAmount, Length: 116201, dtype: float64